In [13]:
import cfbd
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'JOo7NPQiRwoWx1kDmv1VDiujh+ra/nfYLRGy0TVt2KhLM+Yiu6BpXOC7W4YtjBdq'
configuration.api_key_prefix['Authorization'] = 'Bearer'

In [4]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))

In [7]:
def fetch_games_data_for_years(start_year, end_year):
    dataframes = {}
    for year in range(start_year, end_year + 1):
        games = api_instance.get_games(year=year)
        big_12_games = [game for game in games if game.home_conference == 'Big 12' or game.away_conference == 'Big 12']
        
        game_data = pd.DataFrame.from_records(
            [dict(game_id=game.id,
                  year=game.season,
                  home_team=game.home_team,
                  away_team=game.away_team,
                  home_points=game.home_points,
                  away_points=game.away_points) for game in big_12_games]
        )
        dataframes[year] = game_data
    return dataframes

years_data = fetch_games_data_for_years(2016, 2020)


In [8]:
print(years_data)

{2016:       game_id  year       home_team               away_team  home_points  \
0   400868876  2016          Baylor      Northwestern State           55   
1   400869094  2016        Stanford            Kansas State           26   
2   400869507  2016         Houston                Oklahoma           33   
3   400868976  2016   West Virginia                Missouri           26   
4   400869619  2016  Oklahoma State  Southeastern Louisiana           61   
..        ...   ...             ...                     ...          ...   
70  400869604  2016    Kansas State                  Kansas           34   
71  400869596  2016      Iowa State           West Virginia           19   
72  400869611  2016             TCU            Kansas State            6   
73  400869618  2016        Oklahoma          Oklahoma State           38   
74  400868886  2016   West Virginia                  Baylor           24   

    away_points  
0             7  
1            13  
2            23  
3       

In [9]:
combined_data = pd.concat(years_data.values(), ignore_index=True)


In [10]:
def calculate_point_differential(df):
    df['point_differential'] = df['home_points'] - df['away_points']

calculate_point_differential(combined_data)
